# Exploring Chat Templates with SmolLM2 and Llama 3.2

This notebook demonstrates how to use chat templates with the `SmolLM2` and `Llama 3.2` models. Chat templates help structure interactions between users and AI models, ensuring consistent and contextually appropriate responses.

In [1]:
# Import necessary libraries
from transformers import AutoModelForCausalLM, AutoTokenizer
from trl import setup_chat_format
import torch

/Users/tylerhouchin/Desktop/code/smol-course/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## SmolLM2 Chat Template

Let's explore how to use a chat template with the `SmolLM2` model. We'll define a simple conversation and apply the chat template.

In [2]:
# Dynamically set the device
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"

model_name = "HuggingFaceTB/SmolLM2-135M"
model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path=model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)
model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)

In [4]:
# Define messages for SmolLM2
messages = [
    {"role": "user", "content": "Hello, how are you?"},
    {
        "role": "assistant",
        "content": "I'm doing well, thank you! How can I assist you today?",
    },
]

# Apply chat template without tokenization

The tokenizer represents the conversation as a string with special tokens to describe the role of the user and the assistant.


In [5]:
input_text = tokenizer.apply_chat_template(messages, tokenize=False)

print("Conversation with template:", input_text)

Conversation with template: <|im_start|>user
Hello, how are you?<|im_end|>
<|im_start|>assistant
I'm doing well, thank you! How can I assist you today?<|im_end|>



# Decode the conversation

Note that the conversation is represented as above but with a further assistant message.


In [6]:
input_text = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True)

print("Conversation decoded:", tokenizer.decode(token_ids=input_text))

Conversation decoded: <|im_start|>user
Hello, how are you?<|im_end|>
<|im_start|>assistant
I'm doing well, thank you! How can I assist you today?<|im_end|>
<|im_start|>assistant



# Tokenize the conversation

Of course, the tokenizer also tokenizes the conversation and special token as ids that relate to the model's vocabulary.



In [7]:
input_text = tokenizer.apply_chat_template(messages, add_generation_prompt=True)

print("Conversation tokenized:", input_text)

Conversation tokenized: [1, 4093, 198, 19556, 28, 638, 359, 346, 47, 2, 198, 1, 520, 9531, 198, 57, 5248, 2567, 876, 28, 9984, 346, 17, 1073, 416, 339, 4237, 346, 1834, 47, 2, 198, 1, 520, 9531, 198]


<div style= padding: 10px; border-radius: 5px; margin-bottom: 20px'>
    <h2>Exercise: Process a dataset for SFT</h2>
    <p>Take a dataset from the Hugging Face hub and process it for SFT. </p>
    <p><b>Difficulty Levels</b></p>
    <p>🐢 Convert the `HuggingFaceTB/smoltalk` dataset into chatml format.</p>
    <p>🐕 Convert the `openai/gsm8k` dataset into chatml format.</p>
</div>

In [8]:
from IPython.core.display import display, HTML

display(
    HTML("""<iframe
  src="https://huggingface.co/datasets/HuggingFaceTB/smoltalk/embed/viewer/all/train?row=0"
  frameborder="0"
  width="100%"
  height="360px"
></iframe>
""")
)

/var/folders/yj/9j0gvmw94m78t21q_rkctb340000gn/T/ipykernel_77584/1790625692.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython.display
  from IPython.core.display import display, HTML


In [26]:
from datasets import load_dataset

ds = load_dataset("HuggingFaceTB/smoltalk", "everyday-conversations", split="train")
print(ds[0])

{'full_topic': 'Travel/Vacation destinations/Beach resorts', 'messages': [{'content': 'Hi there', 'role': 'user'}, {'content': 'Hello! How can I help you today?', 'role': 'assistant'}, {'content': "I'm looking for a beach resort for my next vacation. Can you recommend some popular ones?", 'role': 'user'}, {'content': "Some popular beach resorts include Maui in Hawaii, the Maldives, and the Bahamas. They're known for their beautiful beaches and crystal-clear waters.", 'role': 'assistant'}, {'content': 'That sounds great. Are there any resorts in the Caribbean that are good for families?', 'role': 'user'}, {'content': 'Yes, the Turks and Caicos Islands and Barbados are excellent choices for family-friendly resorts in the Caribbean. They offer a range of activities and amenities suitable for all ages.', 'role': 'assistant'}, {'content': "Okay, I'll look into those. Thanks for the recommendations!", 'role': 'user'}, {'content': "You're welcome. I hope you find the perfect resort for your v

In [33]:
def process_dataset_entry(sample, drop_full_topic=False):
    # we will use the 'full_topic' to create a system msg to prime the model
    system_message = {
        "content": f"You are a helpful assistant having a conversation about {sample['full_topic']}.",
        "role": "system",
    }
    # prepend the system message to the messages
    messages = [system_message] + sample["messages"]
    # convert the messages into chatml format
    messages = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True)
    sample["messages"] = messages
    if drop_full_topic:
        sample.pop("full_topic")
    return sample

In [34]:
print(f"original sample: {ds[0]}")
print(f"processed sample: {process_dataset_entry(ds[0])}")

original sample: {'full_topic': 'Travel/Vacation destinations/Beach resorts', 'messages': [{'content': 'Hi there', 'role': 'user'}, {'content': 'Hello! How can I help you today?', 'role': 'assistant'}, {'content': "I'm looking for a beach resort for my next vacation. Can you recommend some popular ones?", 'role': 'user'}, {'content': "Some popular beach resorts include Maui in Hawaii, the Maldives, and the Bahamas. They're known for their beautiful beaches and crystal-clear waters.", 'role': 'assistant'}, {'content': 'That sounds great. Are there any resorts in the Caribbean that are good for families?', 'role': 'user'}, {'content': 'Yes, the Turks and Caicos Islands and Barbados are excellent choices for family-friendly resorts in the Caribbean. They offer a range of activities and amenities suitable for all ages.', 'role': 'assistant'}, {'content': "Okay, I'll look into those. Thanks for the recommendations!", 'role': 'user'}, {'content': "You're welcome. I hope you find the perfect 

In [35]:
processed_ds = ds.map(process_dataset_entry, fn_kwargs={"drop_full_topic": True})
processed_ds[1]



Map: 100%|██████████| 2260/2260 [00:01<00:00, 1961.01 examples/s]


{'messages': [1,
  9690,
  198,
  2683,
  359,
  253,
  5356,
  11173,
  1953,
  253,
  6634,
  563,
  5917,
  31,
  20158,
  259,
  1421,
  31,
  61,
  293,
  11054,
  30,
  2,
  198,
  1,
  4093,
  198,
  26843,
  2,
  198,
  1,
  520,
  9531,
  198,
  19556,
  17,
  1073,
  416,
  339,
  724,
  346,
  1834,
  47,
  2,
  198,
  1,
  4093,
  198,
  57,
  5248,
  3012,
  327,
  4795,
  5042,
  30,
  339,
  1277,
  288,
  1042,
  253,
  725,
  2288,
  28,
  564,
  339,
  5248,
  441,
  2090,
  732,
  339,
  1277,
  288,
  536,
  30,
  2,
  198,
  1,
  520,
  9531,
  198,
  20158,
  259,
  1421,
  416,
  325,
  4739,
  30,
  1812,
  359,
  469,
  1574,
  1954,
  284,
  4694,
  338,
  1124,
  724,
  6260,
  1187,
  634,
  3416,
  47,
  2,
  198,
  1,
  4093,
  198,
  57,
  457,
  1786,
  281,
  8162,
  284,
  2815,
  1891,
  351,
  701,
  30,
  339,
  5248,
  597,
  4854,
  281,
  1380,
  540,
  563,
  940,
  2318,
  30,
  2,
  198,
  1,
  520,
  9531,
  198,
  5195,
  506,
  253,
  1109,

In [ ]:
display(
    HTML("""<iframe
  src="https://huggingface.co/datasets/openai/gsm8k/embed/viewer/main/train"
  frameborder="0"
  width="100%"
  height="360px"
></iframe>
""")
)

In [42]:
ds = load_dataset("openai/gsm8k", "main", split="train")
print(ds[0])

{'question': 'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?', 'answer': 'Natalia sold 48/2 = <<48/2=24>>24 clips in May.\nNatalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.\n#### 72'}


In [43]:
def process_dataset_entry(sample):
    # we will primet the model with a system message about the dataset. it is answers to math problems.
    system_message = {
        "role": "system",
        "content": "You are a helpful assistant that can answer questions about math problems.",
    }
    # the 'question' key will be the user's question and the 'answer' will be the assistant's response.
    messages = [system_message] + [{"role": "user", "content": sample["question"]}] + [{"role": "assistant", "content": sample["answer"]}]
    # convert the messages into chatml format
    messages = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True)
    sample["messages"] = messages
    # drop the question and answer keys
    sample.pop("question")
    sample.pop("answer")
    return sample

processed_ds = ds.map(process_dataset_entry)
processed_ds[1]

Map: 100%|██████████| 7473/7473 [00:03<00:00, 2178.19 examples/s]


{'messages': [1,
  9690,
  198,
  2683,
  359,
  253,
  5356,
  11173,
  338,
  416,
  2988,
  2029,
  563,
  4119,
  1828,
  30,
  2,
  198,
  1,
  4093,
  198,
  71,
  1059,
  38668,
  1885,
  33,
  34,
  354,
  5353,
  327,
  3383,
  672,
  9584,
  30,
  718,
  15955,
  28,
  1041,
  915,
  1250,
  216,
  37,
  32,
  3487,
  282,
  3383,
  672,
  9584,
  30,
  1073,
  1083,
  1250,
  1041,
  5301,
  47,
  2,
  198,
  1,
  520,
  9531,
  198,
  71,
  1059,
  38668,
  216,
  33,
  34,
  31,
  38,
  32,
  446,
  1885,
  33691,
  33,
  34,
  31,
  38,
  32,
  45,
  32,
  30,
  34,
  7791,
  32,
  30,
  34,
  567,
  8427,
  30,
  198,
  23830,
  216,
  37,
  32,
  3487,
  28,
  1041,
  11420,
  216,
  32,
  30,
  34,
  1792,
  216,
  37,
  32,
  446,
  1885,
  33691,
  32,
  30,
  34,
  26,
  37,
  32,
  45,
  33,
  32,
  7791,
  33,
  32,
  30,
  198,
  1229,
  216,
  33,
  32,
  2,
  198,
  1,
  520,
  9531,
  198]}

## Conclusion

This notebook demonstrated how to apply chat templates to different models, `SmolLM2`. By structuring interactions with chat templates, we can ensure that AI models provide consistent and contextually relevant responses.

In the exercise you tried out converting a dataset into chatml format. Luckily, TRL will do this for you, but it's useful to understand what's going on under the hood.